In [ ]:
import base64
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from PIL import Image
from pyproj import Transformer
from rasterio.windows import from_bounds


: 

In [64]:
coords = [35.59570312500001,5.878332109674327,118.21289062500001,39.67337039176565]

## Window approach

In [ ]:
min_lon, min_lat, max_lon, max_lat = coords

transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

# Transform to EPSG:3857
min_x, min_y = transformer.transform(min_lon, min_lat)
max_x, max_y = transformer.transform(max_lon, max_lat)

with rasterio.open("sentinel_r10_cog.tif") as src:
    window = from_bounds(min_x, min_y, max_x, max_y, src.transform)
    red = src.read(4, window=window)
    nir = src.read(5, window=window)

    ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)
    ndvi = np.ma.masked_invalid(ndvi)

# Normalize NDVI values to 0-1 for color mapping
ndvi_normalized = (ndvi + 1) / 2
colormap = plt.get_cmap("RdYlGn")
ndvi_colored = colormap(ndvi_normalized)

ndvi_image = (ndvi_colored[:, :, :3] * 255).astype(np.uint8)
image = Image.fromarray(ndvi_image)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title('NDVI Image')
plt.axis('off')
plt.show()

In [ ]:
with rasterio.open("sentinel_r10_4326_cog.tif") as src:
    print("Overviews:", src.overviews(1))  # Assuming band 1, change as needed
    print("Resolutions:", [src.res[0] / ov for ov in src.overviews(1)])

## Overview approach 

Still misses correct tiles 

In [92]:
def get_overview_level(src, band, desired_zoom_level):
    base_res = src.res[0] 
    
    overviews = src.overviews(band)
    desired_res = base_res / (2 ** desired_zoom_level)
    print("Desired resolution:", desired_res)
    
    overview_level = None
    for i, ov in enumerate(overviews):
        overview_res = base_res / ov
        if overview_res <= desired_res:
            overview_level = i
            break

    if overview_level is None:
        overview_level = len(overviews) - 1  # use highest if none match
    return overview_level

In [ ]:
with rasterio.open("sentinel_r10_cog.tif") as src:
    print("Available Overviews for Band 1:", src.overviews(1))  # Replace 1 with the band number you need
    print("Base Resolution:", src.res)
    for i, ov in enumerate(src.overviews(1)):
        print(f"Overview {i}: Resolution {src.res[0] / ov}, {src.res[1] / ov}")
    print(get_overview_level(src,1,10))

In [102]:
lat, lon =  83.96851, 28.26689
zoom_level = 10

In [103]:
import mercantile 
from rasterio.windows import Window
tile = mercantile.tile(lon, lat, zoom_level)

In [ ]:
print(tile)

## Individual tile approach

In [109]:
def read_tile_from_cog(cog_path, tile, zoom, tile_size=256):
    with rasterio.open(cog_path) as src:
        
        # get pixel coords
        x_pixel = tile.x * tile_size
        y_pixel = tile.y * tile_size
        
        
        window = Window(x_pixel, y_pixel, tile_size, tile_size)
        print(window)
        
        red = src.read(4, window=window, resampling=rasterio.enums.Resampling.nearest)
        nir = src.read(5, window=window, resampling=rasterio.enums.Resampling.nearest)
        
        return red, nir

In [ ]:
cog_path = "sentinel_r10_cog.tif"
red, nir = read_tile_from_cog(cog_path, tile, zoom_level)
print(f"Red band shape: {red.shape}, NIR band shape: {nir.shape}")

In [ ]:
# plot red
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(red, cmap='Reds')
plt.title('Red Band')
# plt.colorbar()

# plot nir
plt.subplot(1, 2, 2)
plt.imshow(nir, cmap='Greys')
plt.title('NIR Band')
# plt.colorbar()

# show
plt.show()

In [106]:
def calculate_ndvi(red, nir):
    ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)
    ndvi = np.ma.masked_invalid(ndvi)  # Mask invalid values
    return ndvi

ndvi = calculate_ndvi(red, nir)

## Rio - Tiler Approach

In [201]:
import mercantile
from rio_tiler.io import COGReader

In [ ]:
lat, lon =28.202082, 83.987222
zoom_level = 10
tile = mercantile.tile(lon, lat, zoom_level)
print(tile)
tile_bounds = mercantile.bounds(tile)
print(tile_bounds)

In [219]:
cog_path = "sentinel_r10_cog.tif"

In [ ]:
with COGReader(cog_path) as cog:
    print(cog.crs)
    print(cog.bounds)
    print(cog.minzoom)
    print(cog.maxzoom)
    print(cog.tms)
    tile_data,mask = cog.tile(tile.x,tile.y, zoom_level)
    print(tile_data.shape)
    

In [ ]:

r = tile_data[1]
g = tile_data[2]
b = tile_data[3]
nir = tile_data[4]
# Normalize the 16-bit data to the [0, 1] range for visualization
r_norm = (r - np.min(r)) / (np.max(r) - np.min(r))
g_norm = (g - np.min(g)) / (np.max(g) - np.min(g))
b_norm = (b - np.min(b)) / (np.max(b) - np.min(b))


ndvi = (nir.astype(float) - r.astype(float)) / (nir + r)
ndvi = np.ma.masked_invalid(ndvi)
ndvi_normalized = (ndvi + 1) / 2

rgb = np.stack((r_norm, g_norm, b_norm), axis=-1)

plt.figure(figsize=(10, 10))
plt.imshow(rgb)
plt.title('RGB Composite')
plt.show()


plt.figure(figsize=(10, 10))
plt.imshow(ndvi_normalized)
plt.title('NDVI')
plt.show()